In [ ]:
from time import time
from copy import deepcopy

with open('tasks/16.txt', 'r') as f:
    data = f.read().splitlines()

In [ ]:
class Node:
    def __init__(self, name, rate, children):
        self.name = name
        self.rate = rate
        self.children = children
    def __str__(self):
        return f'Name: {self.name}, rate: {self.rate}, children: {[x.name for x in self.children]}'

In [ ]:
rooms = {}
for line in data:
    room = line.split()[1]
    rate = int(line.split('rate=')[1].split(';')[0])
    if room not in rooms:
        rooms[room] = Node(room, rate, set())
    else:
        rooms[room].rate = rate
    children = line.split('valve')[-1]
    if children.startswith('s'):  
        children = children[2:]
    else:
        children = children[1:]
    children = children.split(', ')
    
    for child in children:
        if child not in rooms:
            temp = Node(child, -1, set())
            rooms[child] = temp
        rooms[room].children.add(rooms[child])

In [ ]:
rooms_with_valves = [x for x in rooms if rooms[x].rate > 0]
rooms_with_valves.append('AA')

In [ ]:
def find_path(node, visited, path_len):
    for child in node.children:
        if child.name not in visited:
            len_dict[child.name] = min(len_dict[child.name], len_dict[node.name] + 1)
    visited.append(node.name)
    return path_len + 1

In [ ]:
room_to_rooms = {}
for r in rooms_with_valves:
    visited = []
    path_len = 0
    len_dict = {x: 100500 for x in rooms}
    len_dict[r] = 0

    not_empty = sorted([x for x in len_dict if x not in visited], key=lambda x: len_dict[x])
    while not_empty:
        path_len = find_path(rooms[not_empty[0]], visited, path_len)
        not_empty = sorted([x for x in len_dict if x not in visited], key=lambda x: len_dict[x])
    
    temp_dict = {}
    for k in len_dict:
        if k in rooms_with_valves:
            temp_dict[k] = len_dict[k]
    room_to_rooms[r] = temp_dict.copy()

In [9]:
def find_max_value_pt_1(start, score, time, long_way, way_target):
    prev_path = deepcopy(start)    
    time -= long_way

    if time <= 0:
        return score
    
    rate = rooms[way_target].rate * (way_target not in prev_path)
    prev_path.add(way_target)
    score += time * rate
    time -= 1

    need_check = [(x, y) for x, y in room_to_rooms[way_target].items() if x not in prev_path]
    all_new_pathes = 0
    for t, w in need_check:
        all_new_pathes = max(all_new_pathes, find_max_value_pt_1(prev_path, score, time, w, t))
    return max(all_new_pathes, score)

def find_max_value_pt2(start, score, time, long_way_1, long_way_2, way_target_1, way_target_2):
    prev_path = deepcopy(start)
    way = min(long_way_1, long_way_2)
    time -= way
    long_way_1 -= way
    long_way_2 -= way
    
    if time <= 0:
        return score

    rate_1 = rate_2 = 0
    if not long_way_1:
        rate_1 = rooms[way_target_1].rate * (way_target_1 not in prev_path)
        prev_path.add(way_target_1)
    if not long_way_2:
        rate_2 = rooms[way_target_2].rate * (way_target_2 not in prev_path)
        prev_path.add(way_target_2)

    score += time * rate_1 * (long_way_1 == 0)
    score += time  * rate_2 *(long_way_2 == 0)
        
    time -= 1
    long_way_1 -= 1
    long_way_2 -= 1

    need_check_1 = []
    if long_way_1 < 0:
        need_check_1 = [(x, y) for x, y in room_to_rooms[way_target_1].items() if x not in prev_path]
    else:
        need_check_1 = [(way_target_1, long_way_1)]
    if long_way_2 < 0:
        need_check_2 = [(x, y) for x, y in room_to_rooms[way_target_2].items() if x not in prev_path]
    else:
        need_check_2 = [(way_target_2, long_way_2)]
    all_new_pathes = 0
    for t_1, w_1 in need_check_1:
        for t_2, w_2 in need_check_2:
            all_new_pathes = max(all_new_pathes, find_max_value_pt2(prev_path, score, time, w_1, w_2, t_1, t_2))
    return max(all_new_pathes, score)

In [11]:
start = time()
print(find_max_value_pt_1({'AA'}, 0, 30, 0, 'AA'))
print(time()-start)

2080
11.28762674331665


In [ ]:
start = time()
part_2_ans = find_max_value_pt2({'AA'}, 0, 26, 0, 0, 'AA', 'AA')

In [18]:
# Second part took about two days :)))
# Optimization high lvl

In [17]:
part_2_ans

2752